<a href="https://colab.research.google.com/github/misogynyX/Data_reclassification/blob/master/(3)news_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
# 4/0AGWRjPNN6oSJJpYQR_5Jj7ssUkHKpuB7cuCFRjapsXRFU5gcIvS_AI
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Hugging Face의 트랜스포머 모델을 설치
!pip install transformers

In [0]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 60 (delta 23), reused 20 (delta 5), pack-reused 0
Unpacking objects: 100% (60/60), done.
/content/Mecab-ko-for-Google-Colab/Mecab-ko-for-Google-Colab
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2020-06-09 09:58:51--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 18.205.93.1, 18.205.93.0, 18.205.93.2, ...
Connecting to bitbucket.org (bitbucket.org)|18.205.93.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz?Signature=Sdb6Y44NjhU48RaNOnuncdMll34%3D&Expires=1591698532&AWSAccessKeyId=AKIA6KOSE3BN

In [0]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [0]:
import urllib.request
import pandas as pd
from konlpy.tag import Mecab
from nltk import FreqDist
import numpy as np
import matplotlib.pyplot as plt

In [0]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [0]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [0]:
# 판다스로 훈련셋과 테스트셋 데이터 로드
train = pd.read_csv("/content/drive/My Drive/Colab Notebooks/NLP_again_project/train_data.csv")
test = pd.read_csv("/content/drive/My Drive/Colab Notebooks/NLP_again_project/test_data.csv")

print(train.shape)
print(test.shape)

(2972, 2)
(1275, 2)


In [0]:
len(train[train['label'] == 1])

569

In [0]:
# 리뷰 문장 추출
sentences = train['text']
# sentences_M = train['text']
sentences[:10]

0    삶과 추억 추억의 마구왕 독고탁 아빠하늘로 은퇴독고탁 캐릭터로 유명한 만화가 이상무...
1    새해 소망 빌어요 울산 간절곶 해맞이 인파 인산인해앵커 시청자 여러분 안녕하십니까 ...
2    품성은 금메달 아냐사재혁 후배 전치 6주 폭행 앵커멘트 2008년 베이징올림픽 남자...
3    독사에 물리고도 공연 이어가 인니 profession여가수profession 끝내 ...
4    직업계 고교생 30로 늘려 대학진학률 낮춘다2022년까지 특성화고교마이스터고교 등 ...
5    대통령 4년차새해 국정운영 방향은경제 활성화일자리 창출뉴시스정일환 기자  박근혜 대...
6    스마트폰이 되고 싶은 아이스마트폰을 놓지 못하는 부모서울신문 나우뉴스 엄마 아빠가 ...
7    모바일 신인류 k세대 백마디 말보다 이모티콘 하나상형문자 세대 1 남자  집에 잘 ...
8    인도서 흑인 gender여대생gender 몰매봉변 당해외교문제 비화뉴델리연합뉴스 나...
9    뉴스 따라잡기 골프채가 부러질 때까지 이번엔 demonize악마demonize 선배...
Name: text, dtype: object

In [0]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 삶과 추억 추억의 마구왕 독고탁 아빠하늘로 은퇴독고탁 캐릭터로 유명한 만화가 이상무본명 박노철씨가 3일 오전 심장마비로 별세했다 71세 고인은 최근 매일 새벽 서울 공덕동 작업실로 출근하며 가수 전인권씨와의 협력작업과 만화 단행본 발간을 준비 중이었다고 한다 평소 술담배를 하지 않고 건강을 챙겼으나 이날 갑자기 세상을 떠 주변을 안타깝게 했다 경북 김천에서 태어나 김천농고를 졸업한 고 [SEP]',
 '[CLS] 새해 소망 빌어요 울산 간절곶 해맞이 인파 인산인해앵커 시청자 여러분 안녕하십니까 1월 1일 금요일 jtbc 뉴스 아침앤입니다 희망찬 2016년 첫날 아침 어떤 모습으로 맞고 계신지 궁금합니다 새해에도 복 많이 받으시고요 가정에 늘 건강과 행복이 깃들길 기원하겠습니다 저희 아침앤은 올해도 생생한 뉴스 진실된 뉴스로 시청자 여러분을 매일 아침 맞이하겠습니다 올 한해 계속해서 많은 사랑과 [SEP]',
 '[CLS] 품성은 금메달 아냐사재혁 후배 전치 6주 폭행 앵커멘트 2008년 베이징올림픽 남자 역도 금메달리스트였던 사재혁 선수가 후배를 폭행해 물의를 빚고 있습니다품성은 금메달이 아니었나 봅니다김순철 기자가 보도합니다  기자 한국 남자 역도 최중량급 기대주인 20살 황우만 씨입니다 얼굴 곳곳에 피멍 자국이 선명하고 왼쪽 눈가의 뼈가 부서져 내려 앉았습니다 올림픽 금메달리스트이자 학교 선배 [SEP]',
 '[CLS] 독사에 물리고도 공연 이어가 인니 profession여가수profession 끝내 사망 앵커  인도네시아 유명 profession여가수profession가 공연 중에 코브라에 물렸습니다 45분 뒤에 무대에서 숨졌는데요 물린 뒤에도 의연한 모습 잃지 않고 해독제 미룬 채 계속 노래했던 게 문제였습니다 서민수 특파원입니다  리포트  인도네시아의 대중가요인 당둣을 부르며 킹코브라와 춤을 추는 29살의 유명 여가수 이르마 불레 인도네시아 서자바섬의 한 마 [SEP]',
 '[CLS] 직업계 고교생 30로 늘려 대학진학률 낮춘다202

In [0]:
# 라벨 추출
labels = train['label'].values
labels

array([0, 0, 0, ..., 0, 0, 0])

### 토크나이징


In [0]:
def Mecab_tok(sentence):
  # 불용어 정의
  stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','속보','특보','기자','trivialize','demonize','molka','porn','abuse','metoo','bearing','gender','profession']
  # 사용 언어 정의 
  userWords = ['일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드']
  
  # Mecab으로 문장을 토큰으로 분리
  tokenizer = Mecab()

  ### train 사용 코드
  tokenized_texts=['[CLS]']
  temp = []
  temp = tokenizer.morphs(sentence)
  temp = [word for word in temp if not word in stopwords]
  tokenized_texts.extend(temp)
  tokenized_texts.append('[SEP]')

  return tokenized_texts

# tokenized_texts = Mecab_tok(sentences_M)

In [0]:
# 불용어 정의
stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','속보','특보','기자','trivialize','demonize','molka','porn','abuse','metoo','bearing','gender','profession']
# 사용 언어 정의 
userWords = ['일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드']

In [0]:
#Bert Tocknizer 로 문장을 토큰화하기
def BertTok(sentences):
  # 불용어 정의
  stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','속보','특보','기자','trivialize','demonize','molka','porn','abuse','metoo','bearing','gender','profession']
  # 사용 언어 정의 
  userWords = ['일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드']

  tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
  tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
  tokenized_texts= [word for word in tokenized_texts if not word in stopwords]
  return tokenized_texts

tokenized_texts = BertTok(sentences)

print (sentences[0])
print (tokenized_texts[0])

[CLS] 삶과 추억 추억의 마구왕 독고탁 아빠하늘로 은퇴독고탁 캐릭터로 유명한 만화가 이상무본명 박노철씨가 3일 오전 심장마비로 별세했다 71세 고인은 최근 매일 새벽 서울 공덕동 작업실로 출근하며 가수 전인권씨와의 협력작업과 만화 단행본 발간을 준비 중이었다고 한다 평소 술담배를 하지 않고 건강을 챙겼으나 이날 갑자기 세상을 떠 주변을 안타깝게 했다 경북 김천에서 태어나 김천농고를 졸업한 고 [SEP]
['[CLS]', '삶', '##과', '추', '##억', '추', '##억', '##의', '마', '##구', '##왕', '독', '##고', '##탁', '아', '##빠', '##하', '##늘', '##로', '은', '##퇴', '##독', '##고', '##탁', '캐', '##릭', '##터', '##로', '유', '##명한', '만', '##화가', '이상', '##무', '##본', '##명', '박', '##노', '##철', '##씨', '##가', '3일', '오', '##전', '심', '##장', '##마', '##비', '##로', '별', '##세', '##했다', '71', '##세', '고', '##인은', '최', '##근', '매', '##일', '새', '##벽', '서울', '공', '##덕', '##동', '작', '##업', '##실', '##로', '출', '##근', '##하며', '가수', '전', '##인', '##권', '##씨', '##와의', '협', '##력', '##작', '##업', '##과', '만', '##화', '단', '##행', '##본', '발', '##간을', '준', '##비', '중', '##이었다', '##고', '한다', '평', '##소', '술', '##담', '##배', '##를', '하지', '않고', '건', '##강', '##을', '챙', '##겼', '##으나', '이', '##날', '갑', '##자', '##기', '세', '##상을', '떠', '주',

In [0]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9409,  11882,   9765,  91837,   9765,  91837,  10459,
         9246,  17196,  40991,   9088,  11664, 119335,   9519, 119008,
        35506, 118762,  11261,   9632, 119362,  80331,  11664, 119335,
         9792,  73352,  21876,  11261,   9625,  68414,   9248, 110817,
        66982,  32537,  40419,  16758,   9319,  28981,  47465,  49212,
        11287,  45367,   9580,  16617,   9491,  13890,  23811,  29455,
        11261,   9353,  24982,  12490,  12513,  24982,   8888,  88236,
         9764,  50248,   9258,  18392,   9415, 118984,  48253,   8896,
       118782,  18778,   9652,  26784,  31503,  11261,   9768,  50248,
        22766, 108399,   9665,  12030,  25347,  49212,  92019,   9981,
        28143,  38709,  26784,  11882,   9248,  18227,   9059,  25549,
        40419,   9323,  90295,   9691,  29455,   9694,  58926,  11664,
        16139,   9926,  22333,   9463, 105462,  76036,  11513,  89093,
        45593,   8865,  47181,  10622,   9743, 118637,  35466,   9638,
      

In [0]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [0]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([   101,   9485,  18623, 119418, 119405,  90833,   9415,  14523,   9766,
         16617,   8907,  51745, 119153,  14523,  12092,   9450, 119199,  11664,
           100,   9313, 119248,  61320,  30842,  34951,   8896,  20626,  25549,
         30005,   9462,  50632,   9576,  11664,   9344,  10739, 119233,  25486,
         33188,  90947,   9638,  54867, 119027,   9891,  46150,  14279,   9485,
         18623, 119418,  45397,  93222,  16323,  40958,  11882,   9378,  17342,
         48446,  22458,  31401,   9935,  90833,  62834,  70672,  10739,   9415,
         14523,   9640,  32159,   9689,  11664, 118965,  24098,   9694,  30873,
          8845,   9487, 118958,  38851,  10739,   9583,   9954,  14523,  35979,
         77039,  64749,  48599,   8870,  96972,   9576,  11664,  12490,  18329,
         45397,  28000,  21386,  21928,   8932,  20595,  12508,   9640,  36553,
         54903,  12638,   9487,  18227,  43022,  25387,  10530,  59355,   9485,
          9689,  40958,  10892,   9665, 

In [0]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [0]:
!nvidia-smi

Tue Jun  9 09:59:35 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    62W / 149W |   6638MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [0]:
# model.load_state_dict(torch.load(f'/content/drive/My Drive/Colab Notebooks/NLP_again_project/best_accuracy.pth'))
# model.eval()

In [0]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 20

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 학습률을 조금씩 감소시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

# 모델 학습

In [0]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [0]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [0]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 20 ========
Training...

  Average training loss: 0.04
  Training epcoh took: 0:01:47

Running Validation...
  Accuracy: 0.89
  Validation took: 0:00:04

======== Epoch 2 / 20 ========
Training...

  Average training loss: 0.04
  Training epcoh took: 0:01:47

Running Validation...
  Accuracy: 0.89
  Validation took: 0:00:04

======== Epoch 3 / 20 ========
Training...

  Average training loss: 0.04
  Training epcoh took: 0:01:47

Running Validation...
  Accuracy: 0.89
  Validation took: 0:00:04

======== Epoch 4 / 20 ========
Training...

  Average training loss: 0.04
  Training epcoh took: 0:01:47

Running Validation...
  Accuracy: 0.89
  Validation took: 0:00:04

======== Epoch 5 / 20 ========
Training...

  Average training loss: 0.04
  Training epcoh took: 0:01:46

Running Validation...
  Accuracy: 0.89
  Validation took: 0:00:04

======== Epoch 6 / 20 ========
Training...

  Average training loss: 0.04
  Training epcoh took: 0:01:46

Running Validation...
  Accu

In [0]:
# torch.save(model.state_dict(), f'/content/drive/My Drive/Colab Notebooks/NLP_again_project/best_accuracy.pth')

# test 데이터셋 생성

In [0]:
# 리뷰 문장 추출
sentences = test['text']
sentences[:10]

0    실험실의 신화 이진아 이은경 소극장 산울림 30년사 한국 연극계의 거목 임영웅80 ...
1    간추린 단신  3대 경제성장 회복법안 처리 뒷받침돼야 외안종범 청와대 경제수석은 올...
2    먼저 옷 벗고 강간당했다 무고50대 여성 집유전주뉴스1 박효익 기자  전주지법 형사...
3    북 새해 방송도 체제선전김정은 특집부터 노래 경연까지 앵커멘트 연말연시 북한 방송은...
4    푸틴의 정적 제거법성관계 molka몰카molka 공개kgb 출신인 푸틴 러시아 대통...
5    서울시민 경제분야 최대 관심사는 체감물가 상승연령대별로 전월세값 상승노후대비 미비로...
6    치솟는 월세에 초선의원 보트 숙박동아일보 영국 런던의 집세를 감당하지 못해 보트에서...
7    이스라엘 총기난사아프간 폭탄테러곳곳 테러 몸살 많은 사람들이 희망과 기대 속에 새해...
8    테러단체 조직원 모집 영상에 도널드 트럼프 등장알샤바브 미국 내 인종차별 증거로 트...
9    포전차공격헬기도발 원점 초토화 앵커멘트 시청자 여러분께서는 어떤 목표를 세우며 새해...
Name: text, dtype: object

In [0]:
# 라벨 추출
labels = test['label'].values
labels

array([0, 0, 1, ..., 1, 0, 0])

In [0]:
def mecab_tok(sentence):
  # 불용어 정의
  stopwords=['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','속보','특보','기자','trivialize','demonize','molka','porn','abuse','metoo','bearing','gender','profession']
  # 사용 언어 정의 
  userWords = ['일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드','일베','메갈리아','워마드']
  
  # Mecab으로 문장을 토큰으로 분리
  tokenizer = Mecab()

  ### train 사용 코드
  tokenized_texts=['[CLS]']
  temp = []
  temp = tokenizer.morphs(sentence)
  temp = [word for word in temp if not word in stopwords]
  tokenized_texts.extend(temp)
  tokenized_texts.append('[SEP]')

  return tokenized_texts

In [0]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
# tokenized_texts = [tokenizer(sent) for sent in sentences]
tokenized_texts = BertTok(sentences)
print (sentences[0])
print (tokenized_texts[0])

실험실의 신화 이진아 이은경 소극장 산울림 30년사 한국 연극계의 거목 임영웅80 극단 산울림 대표와 불문학자 겸 번역가인 부인 오증자77서울여대 명예교수가 10년만 버텨보자며 홍대 근처에 마련한 소극장 산울림이 올해 31주년을 맞았다 누나 임수진53이 극장장 동생 수현51이 예술감독을 맡아 역사를 잇고 있다 연극평론가인 이진아 숙명여자대학교 교수와
['실', '##험', '##실', '##의', '신', '##화', '이', '##진', '##아', '이', '##은', '##경', '소', '##극', '##장', '산', '##울', '##림', '30', '##년', '##사', '한국', '연', '##극', '##계', '##의', '거', '##목', '임', '##영', '##웅', '##80', '극', '##단', '산', '##울', '##림', '대', '##표', '##와', '불', '##문', '##학자', '겸', '번', '##역', '##가', '##인', '부', '##인', '오', '##증', '##자', '##7', '##7', '##서', '##울', '##여', '##대', '명', '##예', '##교', '##수가', '10', '##년', '##만', '[UNK]', '홍', '##대', '근', '##처', '##에', '마', '##련', '##한', '소', '##극', '##장', '산', '##울', '##림', '##이', '올', '##해', '31', '##주', '##년', '##을', '맞', '##았다', '누', '##나', '임', '##수', '##진', '##53', '##이', '극', '##장', '##장', '동', '##생', '수', '##현', '##51', '##이', '예', '##술', '##감', '##독', '##을', '맡', '##아', '역', '##사를', '잇', '##고', '있다', '연', '##극', '##평', '##론', '##가', '##인', 

In [0]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([  9489,  86834,  31503,  10459,   9487,  18227,   9638,  18623,
        16985,   9638,  10892,  31720,   9448,  63243,  13890,   9407,
        78123,  67527,  10244,  10954,  12945,  48556,   9568,  63243,
        21611,  10459,   8863,  68055,   9644,  30858, 119171,  44026,
         8925,  24989,   9407,  78123,  67527,   9069,  37824,  12638,
         9368,  25934, 109522,   8882,   9338,  23160,  11287,  12030,
         9365,  12030,   9580, 119230,  13764,  11305,  11305,  12424,
        78123,  29935,  14423,   9281,  96279,  25242,  73894,  10150,
        10954,  19105,    100,   9992,  14423,   8926,  60469,  10530,
         9246, 101440,  11102,   9448,  63243,  13890,   9407,  78123,
        67527,  10739,   9583,  14523,  10413,  16323,  10954,  10622,
         9256,  27303,   9032,  16439,   9644,  15891,  18623,  67574,
        10739,   8925,  13890,  13890,   9095,  24017,   9460,  30842,
        77581,  10739,   9576,  51945, 105197,  80331,  10622,   9257,
      

In [0]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [0]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([  9489,  86834,  31503,  10459,   9487,  18227,   9638,  18623,  16985,
          9638,  10892,  31720,   9448,  63243,  13890,   9407,  78123,  67527,
         10244,  10954,  12945,  48556,   9568,  63243,  21611,  10459,   8863,
         68055,   9644,  30858, 119171,  44026,   8925,  24989,   9407,  78123,
         67527,   9069,  37824,  12638,   9368,  25934, 109522,   8882,   9338,
         23160,  11287,  12030,   9365,  12030,   9580, 119230,  13764,  11305,
         11305,  12424,  78123,  29935,  14423,   9281,  96279,  25242,  73894,
         10150,  10954,  19105,    100,   9992,  14423,   8926,  60469,  10530,
          9246, 101440,  11102,   9448,  63243,  13890,   9407,  78123,  67527,
         10739,   9583,  14523,  10413,  16323,  10954,  10622,   9256,  27303,
          9032,  16439,   9644,  15891,  18623,  67574,  10739,   8925,  13890,
         13890,   9095,  24017,   9460,  30842,  77581,  10739,   9576,  51945,
        105197,  80331,  10622,   9257, 

In [0]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

## 테스트셋 평가

In [0]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

print("")
print("Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Accuracy: 0.88
Test took: 0:00:17


# 새로운 텍스트 평가

In [0]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    # tokenized_texts = [mecab_tok(sent) for sent in sentences]
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [0]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [0]:
logits = test_sentences(['몰래 카메라에 주변인물 깜짝! 지나가던 사람에게 다친 손을 보여주는 한 남자. 몰카였다는 걸 알고 다함께 웃었습니다'])

print(logits)
print(np.argmax(logits))

[[-0.7421889   0.00711382]]
1


In [0]:
logits = test_sentences(['주연배우가 아깝다. 총체적 난국...'])

print(logits)
print(np.argmax(logits))

[[-0.25396544 -0.6868521 ]]
0


In [0]:
txt_num = 23
test_txt = test['text'][txt_num]
print(test_txt)
print(test['label'][txt_num])

새해 첫 출근길 스모그 주의마스크 준비하세요 날씨는 포근한데 연일 미세먼지가 말썽입니다 특히 건강에 더 해로운 초미세먼지까지 기승인데요 새해 첫 출근길인 내일도 전국 대부분 지역에서는 미세먼지 농도가 높게 나타나겠습니다 김재훈 기자입니다 기자 희뿌연 먼지에 가려진 하늘이 온통 잿빛으로 변했습니다 먼지는 안개에도 엉겨붙어 가시거리는 평소의 절반 수준까지 떨어졌습니다 국내
0


In [0]:
logits = test_sentences([test_txt])

print(logits)
print(np.argmax(logits))

[[ 3.916645  -3.9645553]]
0


In [0]:
import random
a = [random.randint(0,190) for _ in range(10)]
for num in a:
  test_txt = test['text'][num]
  logits = test_sentences([test_txt])
  print(test_txt)
  print(logits)
  aa = test['label'][num] ; bb = np.argmax(logits) 

  if (aa == bb): cc = '네'
  else: cc = "아니오"

  print( 'true label:{}, predicted label:{} 맞췄는가? :::{}'.format(aa,bb,cc) )

  print('-'*40)
  print()

시선 2035 일베가 되어가는 메갈리아소라넷 젊은 남자라면 한 번쯤 들어봤을 국내 최대 성인 사이트다 최근 회원수 100만이라는 소라넷이 남녀 모두의 주목을 받았다 지난 주말 한 방송이 여성의 신상이 노출된 molka몰카molka 동영상 등의 불법성을 집중 추적하면서다 경찰도 전담 수사팀을 꾸렸다 그런데 불똥은 정반대로 튀었다 소라넷 운영진이 방송에서 메갈리아라는 사이트가 주축이 된 소라넷
[[-3.741699   3.7440004]]
true label:1, predicted label:1 맞췄는가? :::네
----------------------------------------

새 희망을 품고하늘에서 본 2016년 첫 날앵커 멘트 어제 하루 많은 분들이 새 희망과 기대를 품고 2016년 새해를 맞이하셨을텐데요 새해 첫 날의 모습을 박민철 기자가 헬기를 타고 둘러봤습니다 리포트 눈 덮인 정상을 향해 힘찬 발걸음을 내딛으며 2016년 첫 하루를 엽니다 정상에 선 시민들은 새해의 정기를 듬뿍 받으며 한해의 각오를 다집니다 물류가 모이는 화물기지에는 휴일이
[[ 3.4486861 -3.3527668]]
true label:0, predicted label:0 맞췄는가? :::네
----------------------------------------

치솟는 월세에 초선의원 보트 숙박동아일보 영국 런던의 집세를 감당하지 못해 보트에서 자취 생활을 시작한 존 머서 영국 보수당 하원의원왼쪽과 기차에서 생활하는 독일 gender여대생gender 레오니 뮐러 씨 사진 출처 텔레그래프 하늘을 뚫을 듯 치솟는 월세에 일개미 같은 삶에 지쳐 나만의 집을 찾는 세계인들의 사연이 화제다 영국 일간 텔레그래프는 2일 보트에서 먹고 자는 초선 의원의 사연을 소개
[[ 3.9141264 -4.0456266]]
true label:0, predicted label:0 맞췄는가? :::네
----------------------------------------

권순홍의 음식기행  홍

# article 을 이용하여 테스트

In [0]:
from copy import copy
import pandas as pd

In [0]:
# csv 파일 불러오기

articles = pd.read_csv("/content/drive/My Drive/Colab Notebooks/NLP_again_project/articles.csv")
articles.head()

,article_id,cp_name,text,authors,keywords,date,url,tags,label
0,20160101000010878,아시아경제,"한민구 국방장관 신년사.. ""군의 존재가치는 싸워 이기는 것""|사랑하는 국군장병과 ...",양낙규,동북아;한미동맹;통일시대;교육훈련;성과,20160101,https://news.v.daum.net/v/20160101000010878,NaN,0
1,20160101000015884,아시아경제,[선거구대란]선거구 실종사태..예비후보자 어떻게 되나| '선거구 실종사태'가 현실화...,유제훈,선거운동;선거구;선거관리위원회;선거구획정;예비후보자,20160101,https://news.v.daum.net/v/20160101000015884,NaN,0
2,20160101000019894,뉴스1,"정의화 ""선거구 비상사태..5일까지 획정안 마련""..직권상정 돌입|(서울=뉴스1) ...",서미선,선거구;직권상정;정의화;선거구획정;20대총선,20160101,https://news.v.daum.net/v/20160101000019894,NaN,0
3,20160101000019895,뉴스1,[총선 D-100]결국 선거구 공백 사태..장기화 가능성도 배제못해| 여야가 20대...,김현,선거구;선거운동;예비후보자;선거관리위원회;총선,20160101,https://news.v.daum.net/v/20160101000019895,NaN,0
4,20160101000021904,경향신문,"전국 246개 선거구, 결국 사라졌다|전국 246개 국회의원 총선 선거구가 사라지는...",김진우;조미덥,선거구;쟁점법안;성폭력범죄;직권상정;여야,20160101,https://news.v.daum.net/v/20160101000021904,NaN,0


In [0]:
def txt_preprocessing(sample):
    sample.drop(['article_id','cp_name','authors','keywords','date','url','tags'],axis = 1, inplace = True)
    sample['text'] = sample['text'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z ]","")
    sample['text'] = sample['text'].str.lower()
    
    return sample

In [0]:
sample = copy(articles)
sample = txt_preprocessing(sample)
sample1 = sample[sample['label']==1]
sample1 = sample1.reset_index(drop=True)

In [0]:
txt_num = 2362
test_txt = sample1['text'][txt_num]
logits = test_sentences([test_txt])

print(logits)
print(np.argmax(logits))
print(test_txt)

[[-0.22120564 -0.7779729 ]]
0
섬마을 학부모들 잇따라 학교 관사 찾아 짐승으로 백기종 전 수서경찰서 강력팀장  김동석 한국교원단체총연합회 대변인 앵커차마 입에 담기도 힘든 끔찍한 일이 작은 섬마을에서 벌어졌죠 전남 신안 섬마을 profession여교사profession 집단 성폭행 사건의 여파가 걷잡을 수 없이 확산되고 있는데요 낙도나 오지로 profession여교사profession 신규발령 내는 걸 가급적 자제하겠다는 대책도 나오는 모양인데 얼마나 효과가 있을지는 물음표입니다 백기
